In [9]:
# Securely load Anthropic API key from local module and set env var if not already set
import os
from secret_key import anthropic_api_key

# Prefer an existing environment variable (for CI/production). Only set it from the local secret when missing.
if not os.getenv("ANTHROPIC_API_KEY"):
    os.environ["ANTHROPIC_API_KEY"] = anthropic_api_key

# Do NOT print the raw key in notebooks or logs. Instead print a confirmation that it's set.
print("ANTHROPIC_API_KEY set?", bool(os.getenv("ANTHROPIC_API_KEY")))

ANTHROPIC_API_KEY set? True


In [10]:
from langchain_core.messages import SystemMessage, HumanMessage

from langchain_anthropic import ChatAnthropic

llm = ChatAnthropic(
    model="claude-3-7-sonnet-20250219",
    temperature=0.6,
    timeout=None,
    max_retries=5,
    anthropic_api_key=anthropic_api_key
)

messages = [
    SystemMessage(content="You are a helpful assistant."),
    HumanMessage(content="What is the capital of France?")
]

response = llm.invoke(messages)
print("Response from Anthropic:")
print(response.content)

Response from Anthropic:
The capital of France is Paris. It's not only the capital city but also the largest city in France, known for its iconic landmarks such as the Eiffel Tower, the Louvre Museum, and Notre-Dame Cathedral.


In [24]:
from langchain_core.prompts import ChatPromptTemplate

# Create a chat template for restaurant naming
name_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at creating unique, sophisticated restaurant names. Your names should be memorable, culturally appropriate, and elegant."),
    ("human", "Name of the country is {country}. Provide ONLY a single name for the restaurant, with no explanation or additional text.")
])

from langchain.chains import LLMChain

name_chain = LLMChain(llm=llm, prompt=name_template, output_key="resturant_name")
# name_chain.run({"country": "Italy"})

In [25]:
# Create a chat template for restaurant naming
menu_template = ChatPromptTemplate.from_messages([
    ("system", "You are an expert at creating unique, sophisticated food item menu list. Your names should be memorable, culturally appropriate, and elegant with the given resturant name."),
    ("human", "Name of the resturant is {resturant_name}. Provide me the ONLY a comma separated fancy food item names for the restaurant menu, with no explanation or additional text."),
])

from langchain.chains import LLMChain

menu_chain = LLMChain(llm=llm, prompt=menu_template, output_key="menu_items")
# menu_chain.run({"resturant_name": "Italy"})

In [26]:
from langchain.chains import SequentialChain

chain = SequentialChain(chains=[name_chain, menu_chain], input_variables=["country"], output_variables=["resturant_name", "menu_items"], verbose=True)
chain.invoke({"country": "Saudi Arabia"})



> Entering new SequentialChain chain...

> Finished chain.


{'country': 'Saudi Arabia',
 'resturant_name': 'Saffron Oasis',
 'menu_items': 'Saffron-Infused Jeweled Rice, Rosewater Pomegranate Lamb Tagine, Cardamom Spiced Date Pudding, Golden Pistachio Baklava, Sumac-Crusted Sea Bass, Harissa Honey Roasted Carrots, Tahini Glazed Aubergine, Preserved Lemon Couscous, Apricot Almond Bisteeya, Saffron Pearl Scallops, Aleppo Pepper Prawns, Ras El Hanout Braised Short Ribs, Fig & Labneh Tartlets, Rosewater Meringue Cloud, Orange Blossom Crème Brûlée, Mint-Scented Lamb Kofta, Pomegranate Molasses Duck, Turmeric Golden Cauliflower, Saffron Poached Pears, Walnut & Halloumi Honey Pastry'}